In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
%%time
df = pd.read_csv('_data_books_main.csv', sep='\t')

df['s'] = df['s'].fillna('')

Wall time: 43.6 s


In [4]:
df.head()

,give_id,reader_id,inventory_num,state__6544.0,state__6545.0,state__6546.0,state__6547.0,state__6548.0,state__6549.0,state__6659.0,age,sample_id,catalog_id,p260c,s
0,1.0,179,03:0000005767,0,1,0,0,0,0,0,31.0,30266.0,5190.0,20.0,семенова мария васильевна год людоеда художес...
1,2.0,179,03:0000005761,0,0,0,0,1,0,0,31.0,30236.0,5185.0,21.0,сказки о солдате библиотека русской сказки худ...
2,3.0,179,03:0000005767,0,1,0,0,0,0,0,31.0,30266.0,5190.0,20.0,семенова мария васильевна год людоеда художес...
3,4.0,179,03:0000005767,0,1,0,0,0,0,0,31.0,30266.0,5190.0,20.0,семенова мария васильевна год людоеда художес...
4,5.0,179,03:0000005767,0,0,0,0,1,0,0,31.0,30266.0,5190.0,20.0,семенова мария васильевна год людоеда художес...


In [5]:
df.loc[df.shape[0], :] = [-1,-1,'',0,0,0,0,0,0,0,0,0,0,0,'']

In [22]:
df.tail()

,give_id,reader_id,inventory_num,state__6544.0,state__6545.0,state__6546.0,state__6547.0,state__6548.0,state__6549.0,state__6659.0,age,sample_id,catalog_id,p260c,s
10418063,11423389.0,110298.0,08:0000227792,1.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,7956437.0,394567.0,19.0,носов николай николаевич приключения незнайки ...
10418064,11423390.0,110298.0,Б08:00001928,1.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,6594976.0,219667.0,2.0,детская энциклопедия справочные материалы
10418065,11423391.0,110298.0,Б08:00027425,1.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,15096635.0,1555329.0,0.0,веселые уроки справочные материалы
10418066,11423392.0,535365.0,08:0001063365,1.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,10470065.0,1069852.0,26.0,филиппов м. а. патриарх никон/м.а.филиппов; бо...
10418067,-1.0,-1.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [32]:
# default number of recommendations
K = 20
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 10

# regularisation for both user and item features
ITEM_ALPHA=1e-5
USER_ALPHA=1e-5

# seed for pseudonumber generations
SEEDNO = 321

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

C:\Users\Yurov\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [10]:
from scipy.sparse import hstack, csr_matrix

In [11]:
%time
tf_idf = joblib.load(open('tfidf01.pkl','rb'))

Wall time: 0 ns


In [12]:
%%time
emb = joblib.load(open('tf_idf_dataframeembeddings', 'rb'))

Wall time: 6.32 s


In [13]:
emb

<10418068x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 70817585 stored elements in Compressed Sparse Row format>

In [23]:
%%time
ii = hstack([csr_matrix(df[['catalog_id', 'state__6544.0', 'state__6545.0', 'state__6546.0', 'state__6547.0', 'state__6548.0', 'state__6549.0', 'state__6659.0', 'p260c']].values), emb])

ii = ii.tocsr()
ii

Wall time: 8.73 s


<10418068x1009 sparse matrix of type '<class 'numpy.float64'>'
	with 101683835 stored elements in Compressed Sparse Row format>

In [24]:
user_feat = csr_matrix(df[['reader_id','age']].drop_duplicates())

user_feat

<450510x2 sparse matrix of type '<class 'numpy.float64'>'
	with 900995 stored elements in Compressed Sparse Row format>

In [25]:
%%time
dataset = Dataset()
dataset.fit(df['reader_id'].unique(), 
            df['catalog_id'].unique())#, 
            #item_features=ii,#item_feat,
            #user_features=user_feat)

Wall time: 1.34 s


In [26]:
%%time
(interactions, weights) = dataset.build_interactions(df[['reader_id','catalog_id']].values)

Wall time: 36.3 s


In [27]:
interactions

<450510x624909 sparse matrix of type '<class 'numpy.int32'>'
	with 10418068 stored elements in COOrdinate format>

In [28]:
uid_map, _, iid_map, _ = dataset.mapping()

In [1]:
import joblib

In [30]:
%%time
joblib.dump(interactions, open('interactions.pkl','wb'))
joblib.dump(weights, open('weights.pkl','wb'))
joblib.dump(uid_map, open('uid_map.pkl','wb'))
joblib.dump(iid_map, open('iid_map.pkl','wb'))


Wall time: 24.7 s


In [2]:
u = joblib.load(open('uid_map.pkl','rb'))

In [4]:
u[-1]

450509

In [33]:
model = LightFM(loss='warp', no_components=NO_COMPONENTS, 
                 learning_rate=LEARNING_RATE, 
                 item_alpha=ITEM_ALPHA,
                 user_alpha=USER_ALPHA,
                 random_state=np.random.RandomState(SEEDNO))

In [35]:
%%time
model.fit(interactions=interactions, verbose=True,
           user_features=user_feat,
           item_features=ii,
           epochs=NO_EPOCHS)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Wall time: 42min 51s


In [37]:
%%time
joblib.dump(model, open('exec/model_book_LFM.pkl', 'wb'))
joblib.dump(ii, open('exec/item_feat.pkl', 'wb'))
joblib.dump(user_feat, open('exec/user_feat.pkl', 'wb'))
joblib.dump(iid_map, open('exec/iid_map.pkl', 'wb'))

Wall time: 21.4 s


In [ ]:
joblib.dump(uid_map, open('exec/uid_map.pkl', 'wb'))

In [77]:
w = model.predict([450509], np.array([v for v in iid_map.values()]), item_features=ii, user_features=user_feat)

In [45]:
idx = (-w).argsort()[:50]

In [48]:
idx

array([226203, 479201, 493318, 578736, 537689, 576471, 289715, 425466,
       153637, 203811, 328497,  80677, 262626, 300764,  32019, 377063,
       269955,  98914,  15912, 400195, 364752, 566938, 106885,   7271,
        15256, 149867, 178930, 344010,  95055, 381340, 286210, 276547,
       476462, 531378, 467817, 461081, 385938, 370082, 124150, 240780,
       180825, 173213,  86229, 308460, 103661, 394162,  96826, 608448,
       150881, 247027], dtype=int64)

In [54]:
iid_map_rev = {v:k for k, v in iid_map.items()}

In [56]:
books = pd.read_csv('data/books_catalog_.csv', sep='\t')

In [63]:
books[books['doc_id']==int(iid_map_rev[i])]

,doc_id,p100a,p245a,p260a,p260b,p260c,p490a,p650a,p084a,p521a
468656,624298,Ключевский Василий Осипович,Сочинения,Москва,Мысль,1988,NaN,История России,63.3(2),NaN


In [71]:
df[df['reader_id']==450509]

,give_id,reader_id,inventory_num,state__6544.0,state__6545.0,state__6546.0,state__6547.0,state__6548.0,state__6549.0,state__6659.0,age,sample_id,catalog_id,p260c,s
6496879,7499041.0,450509.0,07:0000138637,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,11823498.0,256930.0,19.0,линдгрен астрид три повести о малыше и карлсон...
7672061,8674570.0,450509.0,07:0000154508,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,12372996.0,917700.0,8.0,булычев кир путешествие алисы детская классика...
7672062,8674571.0,450509.0,07:0000132413,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,11741510.0,255830.0,19.0,киплинг редьярд маугли золотая библиотека худо...
8437883,9440642.0,450509.0,07:0000154508,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,12372996.0,917700.0,8.0,булычев кир путешествие алисы детская классика...
8437884,9440643.0,450509.0,07:0000132413,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,11741510.0,255830.0,19.0,киплинг редьярд маугли золотая библиотека худо...
8930526,9933407.0,450509.0,07:0000616686,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,11632482.0,356750.0,16.0,губарев виталий георгиевич королевство кривых ...
9721294,10724472.0,450509.0,07:0000616686,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,11632482.0,356750.0,16.0,губарев виталий георгиевич королевство кривых ...
10271366,11275862.0,450509.0,07:0000122843,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,11425000.0,1362.0,12.0,александрова татьяна ивановна домовенок кузька...


In [78]:
for i in idx:
    print(books[books['doc_id']==int(iid_map_rev[i])].values)

[[624298 'Ключевский Василий Осипович' 'Сочинения' 'Москва' 'Мысль'
  '1988' nan 'История России' '63.3(2)' nan]]
[[350067 'Рапацкая Людмила Александровна' 'Русское искусство ХVIII века'
  'Москва' 'Просвещение ; Владос' '1995' 'История мировой культуры'
  'Филология. Искусство и искусствознание' '85' nan]]
[[761227 'Вейнер Эрик' 'География гениальности' 'Москва'
  'Альпина Паблишер' '2017' nan 'Психология' '88' nan]]
[[1627022 'Мортка Марцин' 'Викинг Таппи и первый снег' 'Москва'
  'Вилли Винки ; АСТ' '2019' 'Приключения викинга Таппи'
  'Зарубежная художественная литература для детей и юношества ; Литература для детей и юношества'
  'Д' '0+']]
[[1415277 'Чехов Антон Павлович' 'Полное собрание сочинений и писем'
  'Москва' 'Наука' '1977' nan 'Художественная литература' '84(2Рос=Рус)1'
  nan]]
[[454315 'Золотницкий Давид Иосифович'
  'Академические театры на путях Октября' 'Ленинград' 'Искусство' '1982'
  nan 'Драматический театр' '85.334.3(2)' nan]]
[[976072 nan 'Что такое Библия?' 'М